In [50]:
# Importing necessary modules
import seaborn as sns
import pyspark.sql.functions as F
import pandas as pd
import matplotlib.pyplot as plt
from operator import add
from functools import reduce
import numpy as np
import re
import os
from pyspark.sql.types import StructField, StructType, StringType, LongType
from pyspark.sql.functions import *
import random

# Setting up visualization
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [51]:
indi_full = spark.read.parquet("gs://voter-project-235-25/VM2Uniform--IN--2021-01-15_parq")

NOTE for future: important to add in competitiveness of the election?

In [52]:
cols_to_keep = [
    "Voters_Gender",
    "Voters_Age",
    "Voters_BirthDate",
    "Residence_Families_HHCount",
    "Residence_HHGender_Description",
    "Mailing_Families_HHCount",
    "Mailing_HHGender_Description",

#   !! voter party affiliation
    "Parties_Description", 
    
    "CommercialData_PropertyType",
    "AddressDistricts_Change_Changed_CD",
    "AddressDistricts_Change_Changed_SD",
    "AddressDistricts_Change_Changed_HD",
    "AddressDistricts_Change_Changed_County",
    "Residence_Addresses_Density",
    "CommercialData_EstimatedHHIncome",
    "CommercialData_ISPSA",
    "CommercialData_AreaMedianEducationYears",
    "CommercialData_AreaMedianHousingValue",
    "CommercialData_MosaicZ4Global",
    "CommercialData_AreaPcntHHMarriedCoupleNoChild",
    "CommercialData_AreaPcntHHMarriedCoupleWithChild",
    "CommercialData_AreaPcntHHSpanishSpeaking",
    "CommercialData_AreaPcntHHWithChildren",
    "CommercialData_StateIncomeDecile",
    "Ethnic_Description",
    "EthnicGroups_EthnicGroup1Desc",
    "CommercialData_DwellingType",
    "CommercialData_PresenceOfChildrenCode",
    "CommercialData_PresenceOfPremCredCrdInHome",
    "CommercialData_DonatesToCharityInHome",
    "CommercialData_DwellingUnitSize",
    "CommercialData_ComputerOwnerInHome",
    "CommercialData_DonatesEnvironmentCauseInHome",
    "CommercialData_Education",
    
#   Don't include because of lookahead bias  
#     "Voters_VotingPerformanceEvenYearGeneral",
#     "Voters_VotingPerformanceEvenYearPrimary",
#     "Voters_VotingPerformanceEvenYearGeneralAndPrimary",
#     "Voters_VotingPerformanceMinorElection",
    
#   Other control variables that expect to be highly associated with outcome:
#     "ElectionReturns_P08CountyTurnoutAllRegisteredVoters",
#     "ElectionReturns_P08CountyTurnoutDemocrats",
#     "ElectionReturns_P08CountyTurnoutRepublicans",
    "General_2000",
    "General_2004",
    "PresidentialPrimary_2000",
    "PresidentialPrimary_2004",
        
#   Outcome variable (indiana law happens in 2005, approved by SCOTUS before presidential election in 2008)
    "General_2008"
]


In [53]:
indi = (indi_full
        .select(cols_to_keep))

Based on the voter AGE, calculate the DATE at which they turn 18.
Make a new vaiable that is the YEAR of the earliest election that someone COULD participate in
IF the DATE is less than Nov 3rd, take the year at which they turn 18. If the date is greater than Nov 3rd,
get the year afterwards.

In [57]:
yrs_add = 18
months_add = 18*12

# date of national 
target_month_day_presidential = "11-03"

# date of Indiana's presidential primary
target_month_day_primary = "05-03" 

indi = indi.withColumn("DATE_18", add_months(to_date(col("Voters_BirthDate"),"MM/dd/yyyy"), months_add))
indi.select(["Voters_BirthDate", "DATE_18"]).show(10)
indi = indi.dropna(subset = "Voters_BirthDate")
indi = indi.withColumn("YEAR_18", year("DATE_18"))
indi = indi.withColumn("comparator_date_presidential", to_date(concat(col("YEAR_18"), lit("-"), lit(target_month_day_presidential))))
indi = indi.withColumn("comparator_date_primary", to_date(concat(col("YEAR_18"), lit("-"), lit(target_month_day_primary))))
indi = indi.withColumn("YEAR_ELIGIBLE_TO_VOTE_PRESIDENTIAL", \
                             when(col("DATE_18")<=col("comparator_date_presidential"), col("YEAR_18")) \
                             .otherwise(col("YEAR_18") + 1) \
                            )
indi = indi.withColumn("YEAR_ELIGIBLE_TO_VOTE_PRIMARY", \
                             when(col("DATE_18")<=col("comparator_date_primary"), col("YEAR_18")) \
                             .otherwise(col("YEAR_18") + 1) \
                            )

# check no missing vals:
indi.where(col("YEAR_18").isNull()).select("YEAR_18").show(10)

# get rid of rows where the voter was not old enough to vote in the 2008 general election
indi = indi.filter(col("YEAR_ELIGIBLE_TO_VOTE_PRESIDENTIAL")<=2008).fillna("N", subset = ["General_2008"])

# for the 2000 and 2004 general elections, replace with "N" IF the person was old enough to vote at the time
indi = indi.withColumn("General_2000", \
                      when((col("YEAR_ELIGIBLE_TO_VOTE_PRESIDENTIAL")<= 2004) & \
                           (col("General_2000").isNull()), "N") \
                      .otherwise(col("General_2000")) \
                      )

indi = indi.withColumn("General_2004", \
                      when((col("YEAR_ELIGIBLE_TO_VOTE_PRESIDENTIAL")<= 2004) & \
                           (col("General_2004").isNull()), "N") \
                      .otherwise(col("General_2004")) \
                      )

# do the same for the primaries:
indi = indi.withColumn("PresidentialPrimary_2000", \
                      when((col("YEAR_ELIGIBLE_TO_VOTE_PRIMARY")<= 2004) & \
                           (col("PresidentialPrimary_2000").isNull()), "N") \
                      .otherwise(col("PresidentialPrimary_2000")) \
                      )

indi = indi.withColumn("PresidentialPrimary_2004", \
                      when((col("YEAR_ELIGIBLE_TO_VOTE_PRIMARY")<= 2004) & \
                           (col("PresidentialPrimary_2004").isNull()), "N") \
                      .otherwise(col("PresidentialPrimary_2004")) \
                      )

# make the general voting for 2008 a numeric variable; since we've deleted
# everyone who was not eligible to vote, this can be directly calculated with a 1-0.
indi = indi.withColumn("Voted_General_2008", when(indi.General_2008 == "Y",1).otherwise(0))
indi = indi.drop("General_2008")

+----------------+----------+
|Voters_BirthDate|   DATE_18|
+----------------+----------+
|      05/25/1951|1969-05-25|
|      04/05/1954|1972-04-05|
|      12/04/1959|1977-12-04|
|      08/02/1955|1973-08-02|
|      07/13/1953|1971-07-13|
|      05/26/1961|1979-05-26|
|      06/08/1990|2008-06-08|
|      06/26/1956|1974-06-26|
|      04/15/1985|2003-04-15|
|      03/01/1951|1969-03-01|
+----------------+----------+
only showing top 10 rows



+-------+
|YEAR_18|
+-------+
+-------+



In [58]:
indi.printSchema()

root
 |-- Voters_Gender: string (nullable = true)
 |-- Voters_Age: integer (nullable = true)
 |-- Voters_BirthDate: string (nullable = true)
 |-- Residence_Families_HHCount: integer (nullable = true)
 |-- Residence_HHGender_Description: string (nullable = true)
 |-- Mailing_Families_HHCount: integer (nullable = true)
 |-- Mailing_HHGender_Description: string (nullable = true)
 |-- Parties_Description: string (nullable = true)
 |-- CommercialData_PropertyType: string (nullable = true)
 |-- AddressDistricts_Change_Changed_CD: string (nullable = true)
 |-- AddressDistricts_Change_Changed_SD: string (nullable = true)
 |-- AddressDistricts_Change_Changed_HD: string (nullable = true)
 |-- AddressDistricts_Change_Changed_County: string (nullable = true)
 |-- Residence_Addresses_Density: integer (nullable = true)
 |-- CommercialData_EstimatedHHIncome: string (nullable = true)
 |-- CommercialData_ISPSA: integer (nullable = true)
 |-- CommercialData_AreaMedianEducationYears: integer (nullable = 

In [79]:
# get a subset of the data to prototype code:

sampleind = indi.sample(True, 0.1, seed = 19480384)

In [60]:
sampleind.count()

355879

Go through each of the columns and print the percentage missing or NULL:

In [61]:
num_rows = sampleind.count()
for col_name in sampleind.columns:
    missing_pct = 100*sampleind.filter(col(col_name).isNull()).count() / num_rows
    print(f"{col_name}: {missing_pct}% missing")

Voters_Gender: 0.051983960840622796% missing


Voters_Age: 0.0% missing


Voters_BirthDate: 0.0% missing


Residence_Families_HHCount: 0.0% missing


Residence_HHGender_Description: 0.0% missing


...

comparator_date_presidential: 0.0% missing


target_month_day_primary: 0.0% missing


YEAR_ELIGIBLE_TO_VOTE_PRESIDENTIAL: 0.0% missing


comparator_date_primary: 0.0% missing


YEAR_ELIGIBLE_TO_VOTE_PRIMARY: 0.0% missing
Voted_General_2008: 0.0% missing


In [ ]:
# GET THE UNIQUE VALUES OF EACH COLUMN
# for i in sampleind.columns:
#     print(sampleind.select(i).distinct().collect())
#     print(i)

In [80]:
# Convert CommercialData_EstimatedHHIncome to numeric by just taking the right-most number, and replacing all "$", "-", and "+"

sampleind = sampleind.withColumn("CommercialData_EstimatedHHIncome", regexp_extract(col("CommercialData_EstimatedHHIncome"), "(?<=-).*", 0))

sampleind = sampleind.withColumn("CommercialData_EstimatedHHIncome", \
                             regexp_replace('CommercialData_EstimatedHHIncome', "[\$,+]", "") \
                            )

sampleind = sampleind.withColumn("CommercialData_EstimatedHHIncome",col("CommercialData_EstimatedHHIncome").cast('double'))

In [81]:
sampleind.select(["CommercialData_EstimatedHHIncome"]).show(10, truncate=False)

+--------------------------------+
|CommercialData_EstimatedHHIncome|
+--------------------------------+
|24999.0                         |
|24999.0                         |
|74999.0                         |
|74999.0                         |
|74999.0                         |
|74999.0                         |
|99999.0                         |
|74999.0                         |
|34999.0                         |
|74999.0                         |
+--------------------------------+
only showing top 10 rows



In [82]:
# Convert CommercialData_AreaMedianHousingValue to numeric by replacing "$"

sampleind = sampleind.withColumn("CommercialData_AreaMedianHousingValue", regexp_replace("CommercialData_AreaMedianHousingValue", "\$", ""))
sampleind = sampleind.withColumn("CommercialData_AreaMedianHousingValue",col("CommercialData_AreaMedianHousingValue").cast('double'))
sampleind.select(["CommercialData_AreaMedianHousingValue"]).show(10, truncate=False)

+-------------------------------------+
|CommercialData_AreaMedianHousingValue|
+-------------------------------------+
|139950.0                             |
|139950.0                             |
|172582.0                             |
|172582.0                             |
|140565.0                             |
|140565.0                             |
|139950.0                             |
|172582.0                             |
|172582.0                             |
|172582.0                             |
+-------------------------------------+
only showing top 10 rows



In [86]:
# Search for "Pnct" in the column name, and convert to numeric by replacing "%"

cols_to_convert = [c for c in sampleind.columns if "Pcnt" in c]
cols_to_convert

['CommercialData_AreaPcntHHMarriedCoupleNoChild',
 'CommercialData_AreaPcntHHMarriedCoupleWithChild',
 'CommercialData_AreaPcntHHSpanishSpeaking',
 'CommercialData_AreaPcntHHWithChildren']

In [ ]:
for col_name in cols_to_convert:
    sampleind = sampleind.withColumn(col_name, regexp_replace(col_name, "\%", ""))
    sampleind = sampleind.withColumn(col_name, col(col_name).cast('double'))
    sampleind.select([col_name]).show(5, truncate=False)

In [92]:
# delete columns used for getting voter turnout data

columns_to_drop = ["comparator_date_presidential", "target_month_day_primary", 
                   "YEAR_ELIGIBLE_TO_VOTE_PRESIDENTIAL", "comparator_date_primary", 
                   "YEAR_ELIGIBLE_TO_VOTE_PRIMARY", "YEAR_18", "DATE_18"]

sampleind = sampleind.drop(*columns_to_drop)

In [93]:
sampleind.columns

['Voters_Gender',
 'Voters_Age',
 'Voters_BirthDate',
 'Residence_Families_HHCount',
 'Residence_HHGender_Description',
 'Mailing_Families_HHCount',
 'Mailing_HHGender_Description',
 'Parties_Description',
 'CommercialData_PropertyType',
 'AddressDistricts_Change_Changed_CD',
 'AddressDistricts_Change_Changed_SD',
 'AddressDistricts_Change_Changed_HD',
 'AddressDistricts_Change_Changed_County',
 'Residence_Addresses_Density',
 'CommercialData_EstimatedHHIncome',
 'CommercialData_ISPSA',
 'CommercialData_AreaMedianEducationYears',
 'CommercialData_AreaMedianHousingValue',
 'CommercialData_MosaicZ4Global',
 'CommercialData_AreaPcntHHMarriedCoupleNoChild',
 'CommercialData_AreaPcntHHMarriedCoupleWithChild',
 'CommercialData_AreaPcntHHSpanishSpeaking',
 'CommercialData_AreaPcntHHWithChildren',
 'CommercialData_StateIncomeDecile',
 'Ethnic_Description',
 'EthnicGroups_EthnicGroup1Desc',
 'CommercialData_DwellingType',
 'CommercialData_PresenceOfChildrenCode',
 'CommercialData_PresenceOfPrem

# IMPUTATION OF MISSING VALUES

In [98]:
# for categorical variables, replace missing value with "O" for 
# other (including General_2000, General_2004, PresidentialPrimary_2000, PresidentialPrimary_2004)

# Get a list of all column types:
numeric_cols = [item[0] for item in sampleind.dtypes if item[1].startswith('string')]
categorical_cols = [item[0] for item in sampleind.dtypes if not item[1].startswith('string')]

numeric_cols

# Replace missing values with "O" for other
sampleind = sampleind.fillna("O", subset=categorical_cols)

In [ ]:
# for numeric values, replace missing values with median:

from pyspark.ml.feature import Imputer

# Currently Imputer does not support categorical features and possibly creates incorrect values for a categorical feature.
# https://www.analyticsvidhya.com/blog/2022/05/data-preprocessing-using-pyspark-handling-missing-values/

sampleind = sampleind.select(*(col(c).cast("float").alias(c) for c in sampleind.columns))

imputer = Imputer(
    inputCols=sampleind.columns, 
    outputCols=["{}_imputed".format(c) for c in sampleind.columns]
    ).setStrategy("median")

# Add imputation cols to sampleind
sampleind = imputer.fit(sampleind).transform(sampleind)

Py4JJavaError: An error occurred while calling o2051.fit.
: org.apache.spark.SparkException: surrogate cannot be computed. All the values in Voters_Gender,Voters_BirthDate,Residence_HHGender_Description,Mailing_HHGender_Description,Parties_Description,CommercialData_PropertyType,AddressDistricts_Change_Changed_CD,AddressDistricts_Change_Changed_SD,AddressDistricts_Change_Changed_HD,AddressDistricts_Change_Changed_County,CommercialData_MosaicZ4Global,Ethnic_Description,EthnicGroups_EthnicGroup1Desc,CommercialData_DwellingType,CommercialData_PresenceOfChildrenCode,CommercialData_PresenceOfPremCredCrdInHome,CommercialData_DonatesToCharityInHome,CommercialData_ComputerOwnerInHome,CommercialData_DonatesEnvironmentCauseInHome,CommercialData_Education,General_2000,General_2004,PresidentialPrimary_2000,PresidentialPrimary_2004 are Null, Nan or missingValue(NaN)
	at org.apache.spark.ml.feature.Imputer.fit(Imputer.scala:198)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun

In [ ]:
# scale all data with z score
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler

# Create a VectorAssembler to assemble all numeric columns into a single vector column
assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")

# Apply VectorAssembler to the DataFrame
sampleind_assembled = assembler.transform(sampleind)

# Scale the vector column using StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
scalerModel = scaler.fit(sampleind_assembled)
sampleind_scaled = scalerModel.transform(sampleind_assembled)

# Extract the scaled numeric columns from the scaled vector column
for i, col_name in enumerate(numeric_cols):
    sampleind_scaled = sampleind_scaled.withColumn(col_name + "_scaled", sampleind_scaled.scaled_features[i])

# Drop the original numeric columns and the vector and scaled vector columns
sampleind_scaled = sampleind_scaled.drop(*numeric_cols).drop("features", "scaled_features")

In [35]:
# subset columns by regular expression
def select_matched(df, pattern, printSchema = False, void = False):
    """
    Function that selects the columns of `df` that match the regular expression `pattern`.
    Parameters:
        df -- PySpark dataframe
        pattern: string -- regular expression
        printSchema: bool -- If True, print schema of resulting dataframe
        void: bool -- If True, return None, otherwise, return resulting dataframe
    """
    pattern = re.compile(pattern)
    selected = [s for s in df.columns if pattern.match(s)]
    if (printSchema) & (not void):
        df = df.select(selected)
        df.printSchema()
        return df
    elif printSchema:
        df.select(selected).printSchema()
    elif not void:
        return df.select(selected)

In [ ]:
select_matched(sampleind, "Voters").select("Voters_Gender").show()